In [2]:
import os
import keyring

from typing import Annotated, Literal, List, Optional, TypedDict

from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_openai import ChatOpenAI
from pydantic import BaseModel
from langchain_core.messages import HumanMessage, BaseMessage
from langgraph.prebuilt import create_react_agent

from langchain_community.document_loaders import WebBaseLoader
from langchain_core.tools import tool

# Setup for tools
OPENAI_API_KEY = keyring.get_password('openai', 'key_for_mac')
TAVILY_API_KEY = keyring.get_password('tavily', 'key_for_mac')
os.environ['OPENAI_API_KEY'] = OPENAI_API_KEY
os.environ['TAVILY_API_KEY'] = TAVILY_API_KEY

# Set up LangSmith observability
os.environ['LANGCHAIN_TRACING_V2'] = 'true'
os.environ['LANGCHAIN_ENDPOINT'] = "https://api.smith.langchain.com"
os.environ['LANGCHAIN_API_KEY'] = keyring.get_password('langsmith', 'learning_agent')
os.environ['LANGCHAIN_PROJECT'] = "pr-stupendous-hood-8"

In [3]:
# tavily search tool
tavily_tool = TavilySearchResults(max_results=5)

# Set web search and scraping tools
@tool
def scrape_webpages(urls: List[str]) -> str:
    """ Use requests and bs4 ro scrape the provideed web pages for detailed information. """
    loader = WebBaseLoader(urls)
    docs = loader.load()
    return "\n\n".join(
        [
            f"\n{doc.page_content}\n"
            for doc in docs
        ]
    )
    
# Define the tools
tools = [
    tavily_tool,
    scrape_webpages
]

In [6]:
import functools
import operator

def agent_node(state, agent, name):
    result = agent.invoke(state)
    extracted_contentions = result.get("contentions", [])
    return {
        "messages": [HumanMessage(content=result["messages"][-1].content, name=name)], 
        "contentions": extracted_contentions
    }

llm = ChatOpenAI(model='gpt-4o-mini', temperature=0)

query_system = """
You are a text analysis specialist in a Query Analysis Team.
Your role is to extract contentious issues from the input text that require fact-checking.

Your task:
1. Analyze the user's request or question.
2. Extract contentious issues.
"""

query_agent = create_react_agent(llm, tools=[tavily_tool], state_modifier=query_system)
query_node = functools.partial(agent_node, agent=query_agent, name="John")

In [ ]:
class ContentionAnalysisState(TypedDict):
    messages: Annotated

In [ ]:
# # QueryAnalysisTeam graph state
# class QueryAnalysisTeamState(TypedDict):
#     # A message is added after each team member finishes
#     messages = Annotated[List[BaseMessage], operator.add]
#     # The team members are tracked so they are aware of the other's skill-sets
#     team_members: List[str]
#     # Used to route work. The supervisor calls a function that will update this every time it makes a decision
#     next: str
#     # The list of contention extracted from the query
#     contentions: List[str]